In [5]:
import pandas as pd
import numpy as np

In [30]:
layoffs_df = pd.read_csv('layoffs.csv')

layoffs_df['company']  = layoffs_df['company'].astype('string')
layoffs_df['location']  = layoffs_df['location'].astype('string')
layoffs_df['industry'] = layoffs_df['industry'].astype('string')
layoffs_df['date'] = pd.to_datetime(layoffs_df['date'])
layoffs_df['stage'] = layoffs_df['stage'].astype('string')
layoffs_df['country'] = layoffs_df['country'].astype('string')

layoffs_df = layoffs_df.dropna()
layoffs_df = layoffs_df[layoffs_df['stage'] != 'Unknown']

years = layoffs_df['date'].dt.year
months = layoffs_df['date'].dt.month
month_chunk = (months - 1) // 4
layoffs_df['date_encoded'] = (years - years.min()) * 3 + month_chunk


layoffs_df['date'] = layoffs_df['date_encoded']
layoffs_df = layoffs_df.drop('date_encoded', axis=1)

layoffs_df = layoffs_df.drop('location', axis=1)

stage_mapping = {
    'Seed': 'Early Stage',
    'Series A': 'Early Stage',
    'Series B': 'Growth Stage',
    'Series C': 'Growth Stage',
    'Series D': 'Growth Stage',
    'Series E': 'Growth Stage',
    'Series F': 'Growth Stage',
    'Series G': 'Growth Stage',
    'Series H': 'Growth Stage',
    'Series I': 'Growth Stage',
    'Series J': 'Growth Stage',
    'Post-IPO': 'Post-IPO',
    'Private Equity': 'Post-IPO',
    'Acquired': 'Acquired / Subsidiary',
    'Subsidiary': 'Acquired / Subsidiary',
}

layoffs_df['stage'] = layoffs_df['stage'].map(lambda stage: stage_mapping.get(stage))

layoffs_df = layoffs_df.drop('company', axis=1)

layoffs_df = pd.get_dummies(layoffs_df, drop_first=True, dtype = int)

layoffs_df

,total_laid_off,percentage_laid_off,date,funds_raised,industry_Aerospace,industry_Construction,industry_Consumer,industry_Crypto,industry_Data,industry_Education,...,country_Senegal,country_Seychelles,country_Singapore,country_South Korea,country_Spain,country_Sweden,country_Switzerland,country_United Arab Emirates,country_United Kingdom,country_United States
0,660.0,0.03,11,154.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
7,172.0,0.14,11,392.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8,84.0,0.28,11,272.0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
9,20.0,0.20,11,86.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
17,80.0,0.05,11,54.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3093,4.0,0.20,0,4.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3094,130.0,0.22,0,79.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3095,16.0,1.00,0,6.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3098,6.0,0.75,0,1.0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
